بسم الله

**Troisième extraction**

Troisième notebook. Le but est d'essayer d'extraire tous les livres du site, en une fois, sans passer par des sous catégories comme on l'a fait dans la première étape.

On utilisera cependant la même démarche pour récupérer les urls que dans fiqhqmaliki-extract-1/2.

## Etape 1 : Récupérer les urls

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
##Aller chercher les urls des livres
urls = []
new_urls = []
to_supp =[]
NB_MAX_PAGES = 100
for i in tqdm(range(2,NB_MAX_PAGES)):
    #if i%5 == 0:
    #  print('page ', i, '/', NB_MAX_PAGES, (i/NB_MAX_PAGES)*100, '%')
    i = str(i)
    r = requests.get('https://www.sifatusafwa.com/fr/accueil/?page=' + i)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id = 'products')
    #s = s.find_all('div', class_ = 'products product_list row grid')
    #lines = s.find('h3', class_ = 's_title_block flex_child')
    #Affichage du contenu de la page
    #print(lines)

    #Extraire les liens
    
    for line in s.findAll('a'):
        urls.append(line.get('href'))#line.text.split(\'n') est une liste de chaine de caractères, donc [0] est 
        #seulement le str.
        #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n
    #Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
    urls = list(np.unique(urls))
    #urls
    # On va supprimer les liens qui ne renvoient pas vers des livres : 
    #  - Ceux qui finissent pas '.asc' ou 'desc
    #  - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
    #  - les "javascript"
    
    #Suppression des liens ne renvoyant pas vers des livres.



100%|██████████| 98/98 [10:00<00:00,  6.13s/it]


In [3]:
len(urls)

1284

In [4]:
for link in tqdm(urls):
    if (link.endswith('asc'))==False and \
        (link.endswith('.desc'))==False and \
        len(link.split('/'))>5 and \
        (link.split('/')[5].startswith('?page=')) == False and \
        (link.startswith('https://www.sifatusafwa.com/fr/accueil/?rewrite'))== False and\
        (link.split('/')[5] != ""):
        new_urls.append(link)            
    
for i in range(3):
    try:
        new_urls.remove('javascript:')
        new_urls.remove('javascript:;')
    except ValueError:
        continue
new_urls = pd.DataFrame(new_urls)
new_urls = new_urls.values.T.tolist()[0]
#on enlève tout ce qu'on a rajouté maintenant qu'on a pu faire la comparaison avec ce qu'il y avait
#a supprimer

100%|██████████| 1284/1284 [00:00<00:00, 99831.06it/s]


In [5]:
len(new_urls)

1173

## Etape 2 : Boucle 1 (par ouvrage)

In [6]:
urls = new_urls

#### A régler

In [7]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))
L_string = np.array(L_string)
df = pd.DataFrame((L_string))
df = df.drop_duplicates()#des fois les volumes apparaissent deux fois
df.columns = ['supp', 'categories', str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
df.drop(['supp', 'supp2'], axis = 1, inplace=True)
df.set_index('categories', inplace = True)
urls.remove(urls[0]) #On enlève le lien qu'on a déjà mis dans le dataframe de base


#### Caractéristiques de base

In [8]:
dfs = []
prices = []
auteurs = []
dates = []
domaines =[]
urls_obsolete = []
from tqdm import tqdm  
for url in tqdm(urls):
    # Ajout des caractéristiques du livre (de fiqhmaliki-extract-1)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', class_='product-features')
    L_string = []
    if s !=None:
        for text in s.findAll(class_ ='data-sheet flex_container'):
            L_string.append(text.text.split('\n'))
    else:
        urls_obsolete.append(url)
    L_string = np.array(L_string)
    df_other = pd.DataFrame((L_string))
    df_other = df_other.drop_duplicates()#des fois les volumes apparaissent deux fois
    if len(df_other.columns) == 4:
        df_other.columns = ['supp', 'categories',str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
        df_other.drop(['supp', 'supp2'], axis = 1, inplace=True)
        df_other.set_index('categories', inplace = True)
        dfs.append(df_other)

100%|██████████| 1172/1172 [32:16<00:00,  1.65s/it]


#### Ajout des auteurs/dates de décès/prix/domaines/descriptions 

In [9]:
dfs_2 = []
from tqdm import tqdm  
prices = []
auteurs = []
dates = []
domaines = []
for url in tqdm(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id ='main')
## Ajout des auteurs/dates de décès/prix/domaines/descriptions (de fiqhmaliki-extract-2)
    #On vérifie si l'auteur du livre est disponible:
    s_auteur = s.find("a", class_= 'pro_extra_info_brand') #Si on divise pas en s_auteur et s_prix le find deviendra une liste car ce sera un find d'un find...
    
    #Soit pas d'auteur, soit auteur avec date, soit auteur sans date, soit auteur avec date mais deux
    #fois la parenthèse.

    if  s_auteur != None:#Auteur existe
        s_auteur = s_auteur.text         
        #Le try c'est juste pour couvrir tous les cas de date de décès possible
        try:#Cas 1 : La date existe : on l'ajoute
            s_dateauteur = s_auteur.split('(')[1].split('H)')[0]#s_auteur.split('(') = ['al qadhi iyyadh ', '544H)']
            dates.append(int(s_dateauteur))            

           #Ibn Rushd (Al-Jadd) (520H)	: split('\n') = ['', 'ibn rushd (Al hafid)', '']
        except IndexError:#Cas 2: Pas de date de décès disponible
            dates.append('') #On ne met pas de date pour pas qu'il y ait de décalage: un auteur avec la date de décès de l'auteur 
            #suivant
        except ValueError:#Cas 3: deux fois la parenthèse, il ne peut pas convertir en entier: c'est une 
            #chaine de caractères
            dates.append('') #Avantageux de remplir les auteurs après avoir vérifié la date car ibn rushd (al hafid) est coupé

        auteur = (s_auteur.split('\n')[1].split('(')[0])#s_auteur.split('\n) = ['', 'ibn rushd (Al hafid)', '']
         # ou ['', 'al qadi iyadd','(544)]
        auteurs.append(auteur) #Pour pas qu'on ait un espace à la fin.
            
    else: #Pas d'auteur
        auteurs.append('Auteur non disponible')
        dates.append('') #pour pas qu'il y ait de décalage (exemple un auteur avec la date de décès de l'auteur suivant)
    
    #Ajour du prix    
    s_prix = s.find("span", class_ = 'price')
    try:
        price = s_prix.text
        prices.append(price)

    except AttributeError: #s.find eset une liste et .text n'existe pas pour une liste: normalement plus d'erreur
        print(url, type(s))
        prices.append('Non disponible')
    #Ajout du domaine:
    domaine = url.split('/')[4]
    domaines.append(domaine)

 57%|█████▋    | 667/1172 [19:10<10:54,  1.30s/it]

https://www.sifatusafwa.com/fr/langue-arabe/al-moufrad-al-alam-ahmad-al-hachimi-imlae.html <class 'bs4.element.Tag'>


100%|██████████| 1172/1172 [35:41<00:00,  1.83s/it]


On va faire un DataFrame avec les informations des livres (titre arabe, auteur, édition, volumes... de fiqh-extract1) et un autre avec les auteurs, les domaines.... (de fiqhmaliki-extract-2)

In [10]:
dfs_1 = pd.concat((i for i in dfs), axis = 1)
dfs_2 = pd.DataFrame({"lien": urls, 'auteurs': auteurs, 'date': dates, 'domaine':domaines, 'prix': prices})

In [11]:
dfs_1 = dfs_1.T

In [12]:
dfs_1['lien'] = dfs_1.index
dfs_1.set_index(np.arange(len(dfs_1)), inplace = True)

In [13]:
dfs_2['lien'][0] ==  dfs_1['lien'][0]

True

In [14]:
if 'isbn' in dfs_1.columns and "ean13" in dfs_1.columns:
    dfs_1.drop(['isbn', 'ean13'], axis = 1, inplace=True)#Données propres à l'édition d'un livre. Elles ont du
#etre présentes sur un des liens.

On pourra donc faire une jointure si besoin car les deux dataframes ont les mêmes sites aux mêmes index

On va faire un troisième DataFrame avec les descriptions des livres.

In [16]:
import pickle
with open('file.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(dfs, file)

#### Ajout des descriptions

In [17]:
descriptions = []
for url in tqdm(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id ='main')
    s_description = s.find(class_ = 'product-information')

    try:
        s_description = s_description.text.split('\n')
        description = ""
        passage = False
        for char in range(len(s_description)):
            if s_description[char] != "":
                description = description + s_description[char] + '\n'
                passage = True
            else:
                if passage == True:
                    break
        
    except AttributeError as e :
        print('pas diponible', e)
        description = 'erreur'
    descriptions.append(description)

 57%|█████▋    | 667/1172 [20:16<14:17,  1.70s/it]

pas diponible 'NoneType' object has no attribute 'text'


100%|██████████| 1172/1172 [37:13<00:00,  1.91s/it]


In [18]:
dfs_3 = pd.DataFrame({'description':descriptions, 'lien':urls})
dfs_3.sample(15)

,description,lien
221,Biographie détaillée du Sultan Mamelouk al-Mou...,https://www.sifatusafwa.com/fr/biographies/bio...
449,L'une des plus importantes encyclopédies de Fi...,https://www.sifatusafwa.com/fr/fiqh-généralist...
514,Explication de Dalil at-Talib du Cheikh Mar’i ...,https://www.sifatusafwa.com/fr/fiqh-hanbali/ma...
1153,"Voici une oeuvre rare de l’Imam Ahmad, peu con...",https://www.sifatusafwa.com/fr/tahara-wa-salat...
657,Il s’agit d’un très bon glossaire de l’interpr...,https://www.sifatusafwa.com/fr/kalimat-moufrad...
552,"Explication détaillée et complète, réunie en u...",https://www.sifatusafwa.com/fr/fiqh-rajih-sala...
149,"Bustan Al-Arifin est un livre sur l'ascètisme,...",https://www.sifatusafwa.com/fr/az-zuhd-wa-al-a...
178,L'auteur a commencé par mentionner l'amour pou...,https://www.sifatusafwa.com/fr/biographies/abn...
636,Le Jugement de l’altération de l'Alcool est un...,https://www.sifatusafwa.com/fr/hukm-et-sujets-...
828,Ce recueil est une œuvre impressionnante. Il r...,https://www.sifatusafwa.com/fr/majmu-fatawa-wa...


#### Ajout des éditions

In [19]:
descriptions_edition = []
for url in tqdm(urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id ='main')
    s_editiondescription =  s.find(class_ = 'product-description')
    description = ""
    passage = False
    try:
        if s_editiondescription != None:
            s_editiondescription = s_editiondescription.text.split('\n')
            for char in range(len(s_editiondescription)):
                if s_editiondescription[char] != "":
                    description = description + s_editiondescription[char] + '\n'
                    passage = True
                    char = char + 1
                else:
                    if passage == True:
                        break
                    char = char + 1
    except AttributeError as e:
            description = "Erreur"
    if passage == False: # Pas description d'édition
        description = 'Non disponible'
    descriptions_edition.append(description)


  1%|          | 10/1172 [00:21<40:43,  2.10s/it]


KeyboardInterrupt: 

On ajoute un dernier DataFrame avec les descriptions éditions (certains ouvrages ne possèdent qu'une seule édition donc beaucoup de valeurs nulles si on les met dans le même DataFrame)

In [ ]:
dfs_edition = pd.DataFrame({"descriptions_edition":descriptions_edition, "urls":urls})
dfs_edition["urls"].values

array(['https://www.sifatusafwa.com/fr/adhkar-et-dou-ah/la-roqiyah-shar-iyah-ahmad-at-tayyar.html',
       'https://www.sifatusafwa.com/fr/adhkar-et-dou-ah/sahih-al-adhkar-wa-ar-roqiya-basé-sur-le-tahqiq-d-al-albani-et-moqbil.html',
       'https://www.sifatusafwa.com/fr/al-asma-wa-as-sifat/at-tafwid-fi-as-sifat-al-ilahiyyah-muhammad-ar-rabah.html',
       'https://www.sifatusafwa.com/fr/al-hakimiya-koufr-wal-irjae/al-moubin-fi-qawa-id-al-takfir-al-mou-in-mouhammad-salih-ar-rachid.html',
       'https://www.sifatusafwa.com/fr/al-hakimiya-koufr-wal-irjae/kitab-fi-al-imamah-muhammad-ibn-ali-al-qal-yi-630h.html',
       'https://www.sifatusafwa.com/fr/al-ilm-wa-talab-al-ilm/khazanat-al-ulum-li-amir-al-muminin-bil-andalus-al-mutnassir-billah.html',
       'https://www.sifatusafwa.com/fr/al-ilm-wa-talab-al-ilm/qawa-id-tahqiq-an-nousous-dr-ali-al-imran.html',
       'https://www.sifatusafwa.com/fr/al-iman-ses-piliers-et-ses-branches/les-nobles-angles-entre-les-gens-de-la-sounnah-et-les-égaré

In [24]:
dfs_3.head()

,description,lien
0,Le livre “Al-Iqna’a fi Masa’il al-Ijma’a” de l...,https://www.sifatusafwa.com/fr/accueil/al-iqna...
1,"Cet ouvrage d'Ibn Abi Dounia, parle des djinns...",https://www.sifatusafwa.com/fr/accueil/kitab-h...
2,Explication par l'Imam as-Safarini du poème su...,https://www.sifatusafwa.com/fr/actes-blamables...
3,Risalah du Cheikh Abd-As-Salam ibn Barjas Al-A...,https://www.sifatusafwa.com/fr/actes-blamables...
4,Le livre des grands péchés écrit par le Cheikh...,https://www.sifatusafwa.com/fr/actes-blamables...


In [25]:
dfs_1.to_csv('books_1.csv')
dfs_2.to_csv('books_2.csv')
dfs_3.to_csv('books_3.csv', sep = ";")

#### الحمد لله; la construction du dataset est terminée.

In [ ]:
url = "https://www.sifatusafwa.com/fr/explications-de-livres-de-aqida/charh-aqidah-salaf-wa-ashab-al-hadith-cheikh-abdallah-al-ghunayman.html"
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id ='main')
s_description = s.find(class_ = 'product-information')
s_description

<div class="product-information">
<div class="product-description-short mb-3" id="product-description-short-4006"><div class="rte align_justify" id="short_description_content">
<p>Voici l’explication de la Risalah de l’Imam Abu Uthman as-Sabuni (449H) avec le commentaire éclairé et efficace du Cheikh Abdullah al-Ghunayman, qui figure parmi les ulémas enseignants au Masjid an-Nabawi.</p>
<p>Très belle édition saoudienne.</p>
</div></div>
<div class="steasy_divider between_short_and_price"><div class="steasy_divider_item"></div></div>
<div class="product-actions">
<form action="https://www.sifatusafwa.com/fr/panier" id="add-to-cart-or-refresh" method="post">
<input name="token" type="hidden" value="97ea31bdb5f5f0bca9d418d989214c72"/>
<input id="product_page_product_id" name="id_product" type="hidden" value="4006"/>
<input id="product_customization_id" name="id_customization" type="hidden" value="0"/>
<div class="product-variants"> <div class="product-description">
</div>
</div>
<div clas